In [11]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import pickle


In [12]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to /home/veer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/veer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/veer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))



In [14]:

def preprocess_text(text):
  
    tokens = word_tokenize(text)

    filtered_tokens = set()
    for token in tokens:
        token = token.lower()
        if token.isalnum() and token not in stop_words:
            lemmatized_token = lemmatizer.lemmatize(token)
            filtered_tokens.add(lemmatized_token)
    return ' '.join(filtered_tokens)



In [15]:


df = pd.read_csv('IPC_DATA.csv')


In [16]:
# df.to_csv('IPC_DATA.csv',index=False)

In [17]:
df.head()

,Section,Discription,Bail,Punishment,Triable_By,Congizance,Unnamed: 7,Offence
0,IPC Section 109,"I am reaching out to you concerning [Yashpal],...",Same As Offence,Same As Offence,Same As Offence,Same As Offence,NaN,"Abetment of any offence, if the act abetted is..."
1,IPC Section 110,Whoever abets the commission of an offence sha...,Same As Offence,Same As Offence,Same As Offence,Same As Offence,NaN,"Abetment of any offence, if the person abetted..."
2,IPC Section 111,"I am reaching out to you concerning Yashpal, w...",Same As Offence,Same As Offence,Same As Offence,Same As Offence,NaN,"Abetment of any offence, when one act is abett..."
3,IPC Section 113,"I am reaching out to you concerning Yashpal, w...",Same As Offence,Same As Offence,Same As Offence,Same As Offence,NaN,"Abetment of any offence, when an effect is cau..."
4,IPC Section 114,"I am reaching out to you concerning Yashpal, w...",Same As Offence,Same As Offence,Same As Offence,Same As Offence,NaN,"Abetment of any offence, if abettor is present..."


In [18]:
df=df[~df['Discription'].isnull()]

In [19]:
df.Discription[0]="I hope this letter finds you well. I am writing to seek your urgent assistance in resolving a troubling property dispute that has recently surfaced in our locality. The property in question, situated at [Address of the Property], has been subject to unlawful occupation and disputed ownership by certain individuals. Despite numerous attempts by the rightful owners to reclaim their property through legal means, the situation persists unresolved, causing significant distress and disruption to the community. The actions of these individuals not only violate the rightful ownership of the property but also disrupt the peace and harmony of our neighborhood. It is evident that their behavior is in direct contravention of established laws and regulations governing property rights. Given the gravity of the situation, I urge you to intervene promptly and take necessary legal actions to address this matter. It is imperative that the rights of the rightful owners are upheld and justice is served to restore order and tranquility in our community. I trust in your commitment to upholding the rule of law and ensuring justice for all members of our society. Your prompt attention to this matter is crucial in preventing further escalation of the dispute and resolving it in a fair and equitable manner. Should you require any additional information or assistance in dealing with this matter, please do not hesitate to contact me at your earliest convenience. I am willing to provide any necessary support to facilitate a swift and just resolution to this issue. Thank you for your attention to this urgent matter. I eagerly await your prompt action and resolution of the property dispute."

In [20]:
df.shape

(234, 8)

In [21]:

df['Discription'] = df['Discription'].apply(preprocess_text)



In [22]:

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Discription'])
y = df['Section']



In [23]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [24]:

classifier = MultinomialNB()
classifier.fit(X, y)



MultinomialNB()

In [28]:

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



Accuracy: 0.5957446808510638


In [26]:

def recommend_sections_reverse(Discription, n=2):
    preprocessed_desc = preprocess_text(Discription)
    desc_vectorized = vectorizer.transform([preprocessed_desc])
    

    probas = classifier.predict_proba(desc_vectorized)
    
  
    top_n_indices = np.argsort(probas, axis=1)[:,-n:][:,::-1]
    

    top_n_sections = [classifier.classes_[idx] for idx in top_n_indices[0]]
    
    return top_n_sections



In [27]:

Discription = "I seek urgent assistance with a property dispute at [Address]. Despite legal attempts, unlawful occupation persists, disrupting our community. I urge prompt intervention to uphold the rightful owners' rights and restore order. Please contact me for further details. Thank you for your attention."


recommended_sections_reverse = recommend_sections_reverse(Discription)
print("Recommended Sections (Reverse Order):", recommended_sections_reverse)


Recommended Sections (Reverse Order): ['IPC Section 109', 'IPC Section 213']
